# **Programming Assessment \#5**

Names: ABERIN, Shawn  LIM, Kyle  SINGSON, Raymond

More information on the assessment is found in our Canvas course.

# **Load Pre-trained Embeddings**

*While you don't have to separate your code into blocks, it might be easier if you separated loading / downloading your data from the main part of your solution. Consider placing all loading of data into the code block below.*

In [2]:
import numpy as np
import random
class Vectors:

    def __init__(downloaded,fileloc,type):
        self.loaders = {
        "glove6B":self.load_word_vectors_glove6B
            # TODO add fasttext loading
    }
        self.srcs = {
             "glove6B":"glove6B url here"
        }
        if not downloaded:
            # if not downloaded, depending on type, download the zip then unzip.
                pass
        try:
            self.wordbank,self.vectors = self.loaders[type](fileloc)
        except:
            print("Vector file load error")
                # TODO out this in a dict with less nesting
    def load_word_vectors_glove6B(self,glove_file):
        data = {}
        lines = None
        with open(glove_file, "r", encoding="utf8") as f:
            lines = (line.casefold().split() for line in f)
        # for line in lines:
        #     if line[0].isalnum():
        #         wordslist.append(line[0])
        #         np_line = np.array(line[1:], dtype=np.float32)
        #         vectors.append(np_line)
        #         data.update((line[0],line[1:]))
        try:
            vectors = [(line[0],np.array(line[1:], dtype=np.float32))  for line in lines if line[0].isalnum()]
            data.update(vectors)
            wordslist = data.keys()
            vectors = data.values()
             
            return wordslist, vectors
        except:
            return None, None



# **Your Implementation**

*Again, you don't have to have everything in one block. Use the notebook according to your preferences with the goal of fulfilling the assessment in mind.*

In [ ]:
# def get_similarity(guess, target):
#     #load vector data
#     v1, v2 = vectors[guess], vectors[target]
#     output = np.dot(v1, v2)
#     return abs(round(output.item() * 100, 2))

# def choose_word():
#     #word_bank = listofwords
#     return random.choice(word_bank)

# def play_game():# semantle
# from semantle.data import load_word_vectors
    #i really fucking want to get out of college
    #     #i fucking wanna play genshin
    #     #there is someone cosplaying katheryne in DLSU Lag lib
    #     #there was a cosplay event there and i am here suffering this
    #     #wait i have to do code analysis first
# why did i edit my comments from last week?

import random
import numpy as np
import zipfile as zif
from numpy.linalg import norm
def choose_word():
    word_bank, _ = load_word_vectors_glove6B()
    return random.choice(word_bank)
class semantle:
    
    # vec_class is the type of pre-trained vector to be used. 
    def __init__(self,downloaded = False,type = None, vec = None):
        self.SEN=-2
        self.prev = self.SEN
        self.first_temps = {
            1:"synonyms",
            0:"unrelated",
            -1:"antonyms"
        }
        if vec is None:
            vec = Vectors(ownloaded,type)
        self.word_bank = vec.wordbank
        self.vec = self.vectors #set this to our words= vectors
        self.maketarget()
    def maketarget(self):
        self.w = random.choice(self.word_bank)
        self.vec_w = norm(self.vec[self.w])

    def check(self,guess):
        
        if guess == self.w:
                print(f"{self.w} is correct")
                return True
        try:
            the_temp = lambda s: "colder" if s < self.prev else "warmer"
            vec_guess = self.vec[guess] #in case we need a generic method to access, create one
            sim = np.dot(norm(vec_guess),self.vec_w)
            temp = the_temp(sim) if self.prev  != self.SEN else self.first_temps[round(sim)]
            self.prev = sim
            print(f"{guess} is {temp}, cosine similarity is: {sim} ")
        except:
            print("Word not recognized, please try again")
        return False
guess= ""
game = semantle()
correct = False
step = 0
while not correct :
    print(f"step:{step}")
    guess = input("Enter your guess: ").lower().strip()
    correct = game.check(guess)
print("thank you")
